In [2]:
# Import the dependencies.
import pandas as pd
import gmaps
import requests
%matplotlib inline
# Import the API key.
from config import g_key

In [3]:
# Store the CSV you saved created in part one into a DataFrame.
city_data_df = pd.read_csv("weather_data/cities.csv")
city_data_df.head()

,City_ID,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed
0,0,Billings,US,2020-06-22 00:02:23,45.78,-108.50,78.80,29,75,11.41
1,1,Busselton,AU,2020-06-22 00:02:23,-33.65,115.33,60.01,55,49,22.79
2,2,Rikitea,PF,2020-06-22 00:02:23,-23.12,-134.97,70.79,68,86,18.01
3,3,Port Alfred,ZA,2020-06-22 00:02:24,-33.59,26.89,57.99,93,17,6.73
4,4,Thompson,CA,2020-06-22 00:01:56,55.74,-97.86,60.80,82,75,5.82


In [4]:
#checking data types since gmaps uses only integers or floating point decimal numbers
city_data_df.dtypes

City_ID         int64
City           object
Country        object
Date           object
Lat           float64
Lng           float64
Max Temp      float64
Humidity        int64
Cloudiness      int64
Wind Speed    float64
dtype: object

In [5]:
#CREATING HEATMAPS

# Configure gmaps to use your Google API key.
gmaps.configure(api_key=g_key)

In [7]:
# Heatmap of temperature
# Get the maximum temperature.
max_temp = city_data_df["Max Temp"]
temps = []
for temp in max_temp:
    temps.append(max(temp, 0))
# Get the latitude and longitude.
locations = city_data_df[["Lat", "Lng"]]
# Get the maximum temperature.
max_temp = city_data_df["Max Temp"]
# Assign the figure variable.
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
# Assign the heatmap variable.
heat_layer = gmaps.heatmap_layer(locations, weights=temps, dissipating=False, max_intensity=300, point_radius=4)
# Add the heatmap layer.
fig.add_layer(heat_layer)
# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))

In [8]:
# Heatmap of percent humidity
locations = city_data_df[["Lat", "Lng"]]
humidity = city_data_df["Humidity"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=humidity, dissipating=False, max_intensity=300, point_radius=4)

fig.add_layer(heat_layer)
# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))

In [9]:
# Heatmap of percent cloudiness
locations = city_data_df[["Lat", "Lng"]]
clouds = city_data_df["Cloudiness"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=clouds, dissipating=False, max_intensity=300, point_radius=4)

fig.add_layer(heat_layer)
# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))

In [10]:
# Heatmap of wind speed
locations = city_data_df[["Lat", "Lng"]]
wind_speed = city_data_df["Wind Speed"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=wind_speed, dissipating=False, max_intensity=300, point_radius=4)

fig.add_layer(heat_layer)
# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))

In [11]:
# Ask the customer to add a minimum and maximum temperature value.
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 50
What is the maximum temperature you would like for your trip? 85


In [12]:
# Filter the dataset to find the cities that fit the criteria.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed
0,0,Billings,US,2020-06-22 00:02:23,45.78,-108.50,78.80,29,75,11.41
1,1,Busselton,AU,2020-06-22 00:02:23,-33.65,115.33,60.01,55,49,22.79
2,2,Rikitea,PF,2020-06-22 00:02:23,-23.12,-134.97,70.79,68,86,18.01
3,3,Port Alfred,ZA,2020-06-22 00:02:24,-33.59,26.89,57.99,93,17,6.73
4,4,Thompson,CA,2020-06-22 00:01:56,55.74,-97.86,60.80,82,75,5.82
6,6,Lebu,CL,2020-06-22 00:02:25,-37.62,-73.65,50.56,84,100,14.85
7,7,Airai,TL,2020-06-22 00:02:25,-8.93,125.41,63.16,75,79,4.74
8,8,Georgetown,MY,2020-06-22 00:02:25,5.41,100.34,78.80,100,20,4.76
9,9,Bredasdorp,ZA,2020-06-22 00:02:25,-34.53,20.04,51.80,93,73,4.38
10,10,Ribeira Grande,PT,2020-06-22 00:02:26,38.52,-28.70,66.07,89,3,13.49


In [13]:
#check and see if there are any null values 
preferred_cities_df.isnull().sum()

City_ID       0
City          0
Country       4
Date          0
Lat           0
Lng           0
Max Temp      0
Humidity      0
Cloudiness    0
Wind Speed    0
dtype: int64

In [14]:
#count filled cells
preferred_cities_df.count()

City_ID       478
City          478
Country       474
Date          478
Lat           478
Lng           478
Max Temp      478
Humidity      478
Cloudiness    478
Wind Speed    478
dtype: int64

In [15]:
# Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = preferred_cities_df[["City", "Country", "Max Temp", "Lat", "Lng"]].copy()
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Lat,Lng,Hotel Name
0,Billings,US,78.80,45.78,-108.50,
1,Busselton,AU,60.01,-33.65,115.33,
2,Rikitea,PF,70.79,-23.12,-134.97,
3,Port Alfred,ZA,57.99,-33.59,26.89,
4,Thompson,CA,60.80,55.74,-97.86,
6,Lebu,CL,50.56,-37.62,-73.65,
7,Airai,TL,63.16,-8.93,125.41,
8,Georgetown,MY,78.80,5.41,100.34,
9,Bredasdorp,ZA,51.80,-34.53,20.04,
10,Ribeira Grande,PT,66.07,38.52,-28.70,


In [16]:
# Set parameters to search for a hotel.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}
# Iterate through the DataFrame.
for index, row in hotel_df.iterrows():
    # Get the latitude and longitude.
    lat = row["Lat"]
    lng = row["Lng"]

    # Add the latitude and longitude to location key for the params dictionary.
    params["location"] = f"{lat},{lng}"

    # Use the search term: "lodging" and our latitude and longitude.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    # Make request and get the JSON data from the search.
    hotels = requests.get(base_url, params=params).json()
    # Grab the first hotel from the results and store the name.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not foun

In [22]:
#Info box template
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""
# Store the DataFrame Row.
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
# Add a heatmap of temperature for the vacation spots.
locations = hotel_df[["Lat", "Lng"]]
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp, dissipating=False,
             max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)
# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))